# Process SIMBAD HIP results

Many of these HIP stars are in a binary system, unresolved by Hipparcos but resolved by Gaia which will cause cross-matching issue. Usually by the fact that they are missing `FLUX_G` but not `FLUX_V`, `FLUX_J` because Gaia did resolve them. The underlying issue is these binary stars are closed enough to be observed by 2MASS/Hipparcos as a single source hence having V, JHK band magnitude but resolved by Gaia hence not having a G-band magnitude because such single object does not exist in Gaia.

## Data Loading and cross-match HIP and Gaia

Every HIP stars should have a Gaia counterpart unless they are very bright or not a stars.

*Note*: These HIP ids are not strictly stars so we will delete them
- HIP 1902 is a globular cluster NGC 104
- HIP 59018 is an artifact
- HIP 60450 is an artifact
- HIP 95723 is an artifact
- HIP 103992 is a planetary nebular NGC 7009
- HIP 108802 is a composite object / blend
- HIP 54948 is a cluster of star (cant be sure which one they are referring, doubt Hipparcos has resolved them)
- And a few more...

And this is a list of NASA Scientific Visualization Studio missing naked eyes visible stars (HR 2366 is not actually missing from HIP)

HR 2950 is a binary stars resolved in Gaia, (v) means manually checked Gaia has the stars.

missing_stars = HR + [4210 (v), 4375 (v), 4374 (v), 5978 (v), 5977 (v), 4729 (v), 2322 (v), 5343 (v), 2950 (v+v), 1982 (v), 5034 (v), 4619 (v), 1704 (v), 6660 (v), 6263 (v), 2341 (v), 6848 (v)]

To get Gaia data, the ADQL query used is as follow:

```sql
with x as
(
	SELECT G.source_id, H2.original_ext_source_id as hip, G.ra, G.dec, G.parallax, G.parallax_over_error, G.pmra, G.pmdec, G.phot_g_mean_mag, G.bp_rp, G.radial_velocity, G.radial_velocity_error, G.astrometric_params_solved, G.ruwe, G.rv_expected_sig_to_noise, G.phot_g_mean_mag - (0.01426 * POWER(G.bp_rp, 3) - 0.2156 * POWER(G.bp_rp, 2) + 0.01424 * POWER(G.bp_rp, 1) - 0.02704) as v_mag 
    FROM gaiadr3.gaia_source AS G
	LEFT JOIN gaiadr3.hipparcos2_best_neighbour AS H2 ON H2.source_id = G.source_id
	WHERE G.phot_g_mean_mag IS NOT NULL OR H2.source_id IS NOT NULL
)
SELECT * 
FROM x
WHERE (bp_rp IS NOT NULL AND v_mag <= 15.5) OR (bp_rp IS NULL AND phot_g_mean_mag <= 15.5)
```

correspond to file named `1733546104637O-result.fits` used in here (You will get a different filename if you do the same query yourself)


## First query all possible Hipparcos ID from SIMBAD and dump the whole table locally

In [1]:
import pathlib

import astropy.units as u
import numpy as np
import pandas as pd
import tqdm
from astropy.coordinates import SkyCoord
from astropy.table import Table, vstack
from astropy.time import Time
from astroquery.gaia import Gaia

base_path = pathlib.Path("./simbad_query_results")
gaia_base_path = pathlib.Path("./gaia_query_results")
hip_to_remove = [1902, 54948]

# ==============================================================================
# Cleanup the HIP data from SIMBAD
# ==============================================================================

# Read the HIP data from simbad
data = Table.read(base_path / "hip_simbad.dat", format="ascii")
simbad_df = data.to_pandas()
# Parse a string that seperated by | and get those with HIP and Gaia
simbad_df["IDS_ls"] = simbad_df["IDS"].apply(lambda x: x.split("|"))
# get the HIP ID as integer, only the number without "HIP"
simbad_df["hip"] = simbad_df["IDS_ls"].apply(
    lambda x: int([i.split("HIP")[-1] for i in x if "HIP" in i][0])
    if any("HIP" in i for i in x)
    else 0
)
# delete those row with duplicate HIP ID, keep the first one
simbad_df = simbad_df.drop_duplicates(subset="hip").reset_index(drop=True)
# some are artifact, use apply() in case we want to do something more complex. HIP 1902 is Globular Cluster, not a star
bad_idx = simbad_df["OTYPE_N"].apply(lambda x: ("err" in x)) | np.sum(
    [(simbad_df["hip"] == i) for i in hip_to_remove], axis=0
)
print(
    "These HIP ids does not actually exist! -> ", simbad_df["hip"][bad_idx].to_numpy()
)
# delete the bad ones
simbad_df = simbad_df[~bad_idx].reset_index(drop=True)
# add a column for Gaia DR3 source_id and fill with zero in int64 to prevent overflowing issue
simbad_df["source_id"] = np.zeros(len(simbad_df), dtype=np.int64)
# get the Gaia DR3 ID as integer, only the number part
simbad_df["source_id"] = simbad_df["IDS_ls"].apply(
    lambda x: int([i.split("Gaia DR3")[-1] for i in x if "Gaia DR3" in i][0])
    if any("Gaia DR3" in i for i in x)
    else 0
)

# Some HIP stars are diffult to be matched, so do those manually.
hip_gaia_pairs = {
    116191: 2395220664463237120,
    94223: 6661809427913911296,
    27996: 504043622260138496,
    28442: 2891185070988913536,
    68588: 1508886439891509248,
    107711: 6572227714472220416,
    12706: 2502847975056102400,
    9487: 2517584007848935808,
    95947: 2026116260337482112,
    44471: 1010770873228612608,
    85267: 5922299347569528832,
    1829: 2799042079568358784,
    114222: 2232894437098037888,
    55203: 756853643638639104,
    10826: 2493390319631956352,
    47758: 5657043095061410944,
    35793: 5616197646838713728,
    40167: 657244586015485440,
    73695: 1588936345294751872,
    84012: 4136366183978856960,
    108431: 6413081274731392256,
    80170: 1201067439457715072,
    
}
# put those Gaia source_id manually
for hip, source_id in hip_gaia_pairs.items():
    simbad_df.loc[simbad_df["hip"] == hip, "source_id"] = source_id

# ==============================================================================
# Correct known bad data
# ==============================================================================
# HIP 68702 has very wrong FLUX_B, correct it such that FLUX_B - FLUX_V = –0.23
simbad_df.loc[simbad_df["hip"] == 68702, "FLUX_B"] = (
    simbad_df.loc[simbad_df["hip"] == 68702, "FLUX_V"].values + 0.23
)
# HIP 42913 has no FLUX_V, FLUX_B, see https://adsabs.harvard.edu/pdf/2000IBVS.4999....1O
simbad_df.loc[simbad_df["hip"] == 68702, "FLUX_V"] = 1.96
simbad_df.loc[simbad_df["hip"] == 68702, "FLUX_B"] = 2.00


# ==============================================================================
# Crossmatch with Gaia DR3
# ==============================================================================

# this table contains the Gaia DR3 source_id for the HIP stars
gaia_t = Table.read(gaia_base_path / "1733546104637O-result.fits", format="fits")
gaia_t = gaia_t.to_pandas()

# cross-match those without HIP
matched_hip, idx1, idx2 = np.intersect1d(
    simbad_df["hip"].values,
    gaia_t.loc[gaia_t["hip"] > 0, "hip"].values,
    return_indices=True,
)

simbad_df.loc[idx1, "source_id"] = (
    gaia_t.loc[gaia_t["hip"] > 0, "source_id"].iloc[idx2].values
)

# See how many stars still without Gaia DR3 ID and dim enough to have Gaia DR3 ID
hip_wo_source_id_idx = (simbad_df["source_id"] == 0) & (
    (simbad_df["FLUX_V"].fillna(99.99) > 4.0)
    & (simbad_df["FLUX_J"].fillna(99.99) > 4.0)
)
print("Stars (mag>4.0) without Gaia DR3 ID: ", np.sum((hip_wo_source_id_idx)))

simbad_df.loc[hip_wo_source_id_idx]

These HIP ids does not actually exist! ->  [114110 114176  95723  54948  59018  60450   1902]
Stars (mag>4.0) without Gaia DR3 ID:  925


,MAIN_ID,RA,DEC,RA_PREC,DEC_PREC,COO_ERR_MAJA,COO_ERR_MINA,COO_ERR_ANGLE,COO_QUAL,COO_WAVELENGTH,...,OTYPE_N,SP_TYPE,SP_QUAL,SP_BIBCODE,RV_VALUE,RVZ_ERROR,SCRIPT_NUMBER_ID,IDS_ls,hip,source_id
327,HD 221685,23 34 19.3456,-09 42 34.265,9,9,22.790,13.240,90,A,O,...,**,F2V,C,1999MSS...C05....0H,9.20,2.90,329,"[CCDM J23343-0943AB, ** BU 387, BD-10 6134, ...",116329,0
405,HD 221773,23 35 19.2266,-57 29 42.275,9,9,11.530,9.140,90,A,O,...,**,F3V,C,1975MSS...C01....0H,NaN,NaN,407,"[CCDM J23353-5730AB, ** I 25AB, CD-58 8593,...",116407,0
434,HD 221839,23 35 40.0529,-27 29 25.866,14,14,0.397,0.442,90,A,O,...,**,F7V+G3V,C,1969PASP...81..643C,0.34,1.49,436,"[CCDM J23357-2729AB, ** SEE 492, ADS 16850 AB...",116436,0
526,HD 221982,23 36 57.2023,-36 48 04.805,9,9,12.720,11.330,90,A,O,...,**,F6V,C,1982MSS...C03....0H,-0.80,0.90,528,"[CCDM J23370-3648AB, ** I 693, CD-37 15311, C...",116528,0
660,BD+03 4884,23 38 30.5307,+04 19 00.025,9,9,26.850,26.170,87,A,O,...,**,G5V+K2V,C,2006A&A...460..695T,5.70,NaN,662,"[CCDM J23385+0419AB, ** J 582, ADS 16892 AB, ...",116662,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117546,StKM 1-1927,21 44 26.8658,+06 06 40.874,9,9,66.440,27.990,90,A,O,...,**,K4/5,D,1986AJ.....91..144S,NaN,NaN,1342,"[HIP 107342, ** HDS 3096, HIC 107342, 2MASS J2...",107342,0
117597,G 126-32,21 45 04.9370,+19 53 35.221,9,9,41.510,34.320,90,A,O,...,**,M1.5V,C,2013AJ....145..102L,NaN,NaN,1393,"[HIP 107393, ** RAO 466, ASCC 905796, G 126-...",107393,0
117748,HD 207136,21 46 53.4797,+00 51 02.847,9,9,20.560,10.020,90,A,O,...,**,F2IV/V,D,1999MSS...C05....0H,9.40,8.60,1544,"[CCDM J21469+0051AB, ** STF 2825, ADS 15313 AB...",107544,0
118051,HD 207000,21 51 00.6122,-76 12 23.650,9,9,9.230,8.010,158,A,O,...,**,G8/K0V+A/F,D,1975MSS...C01....0H,16.42,0.61,1847,"[CCDM J21510-7612AB, ** HDS 3112, CD-76 1109,...",107847,0


In [2]:
# Guess where these stars are located at J2016, assume 0 proper motion if not available
c = SkyCoord(
    ra=simbad_df.loc[hip_wo_source_id_idx, "RA_d2000"].values * u.deg,
    dec=simbad_df.loc[hip_wo_source_id_idx, "DEC_d2000"].values * u.deg,
    pm_ra_cosdec=simbad_df.loc[hip_wo_source_id_idx, "PMRA"].fillna(0).values
    * u.mas
    / u.yr,
    pm_dec=simbad_df.loc[hip_wo_source_id_idx, "PMDEC"].fillna(0).values * u.mas / u.yr,
    obstime=Time(2000, format="jyear"),
)
c2 = c.apply_space_motion(Time(2016, format="jyear"))
result_ls = []
# not that many so cone search one by one
for i in tqdm.tqdm(range(np.sum(hip_wo_source_id_idx))):
    result = Gaia.cone_search(c2[i], radius=5 * u.arcsec).get_results()
    # add the HIP ID for reference
    result["hip"] = simbad_df.loc[hip_wo_source_id_idx, "hip"].iloc[i]
    # select the brightest one, because either those stars are too dim to be observed by Hipparcos
    # or they are in binary and HIP refers to the main one anyway
    result = result[result["phot_g_mean_mag"] == np.min(result["phot_g_mean_mag"])]
    expected_v_mag = simbad_df.loc[hip_wo_source_id_idx, "FLUX_V"].iloc[i]
    g_mag = result["phot_g_mean_mag"]
    result_ls.append(result)

# save the result table as a backup
result_table = vstack(result_ls)
# these columns are not needed and cause saving error
# result_table.remove_columns(["DESIGNATION", "phot_variable_flag", "libname_gspphot", "random_index"])
result_table.write(base_path / "hip_wo_sourceid_gaia_crossmatch.dat", format="ascii")

# in the future, you can simply check if the result is [] to manually match HIP to Gaia (probably wont happen in the future)
# for i in range(len(result_ls)):
#     if len(result_ls[i]) == 0:
#         print("No match for HIP ", simbad_df.loc[hip_wo_source_id_idx, "hip"].iloc[i])

c:\Users\Henry\miniconda3\Lib\site-packages\erfa\core.py:133: ErfaWarning: ERFA function "pmsafe" yielded 925 of "distance overridden (Note 6)"
  warn(f'ERFA function "{func_name}" yielded {wmsg}', ErfaWarning)
100%|██████████| 925/925 [11:37<00:00,  1.33it/s]


In [3]:
# Put those Gaia source_id to the main dataframe
# We can get other information later
for i in range(len(result_ls)):
    simbad_df.loc[simbad_df["hip"] == result_ls[i]["hip"][0], "source_id"] = result_ls[
        i
    ]["SOURCE_ID"]

# add component ID, we will deal with binary star in the future
simbad_df["componentid"] = np.zeros(len(simbad_df), dtype=int)
simbad_df.drop(columns=["IDS_ls"], inplace=True)

# save the final table
Table.from_pandas(simbad_df).write(base_path / "hip_processed.dat", format="ascii")

## Now deal with binary and component IDs

You should restart the notebook and run the cells below

In [1]:
import pathlib

import astropy.units as u
import numpy as np
import pandas as pd
from astropy.table import Table, vstack
from astroquery.gaia import Gaia
from astroquery.simbad import Simbad

base_path = pathlib.Path("./simbad_query_results")

# Read the HIP data from above
data = Table.read(base_path / "hip_processed.dat", format="ascii")
simbad_df = data.to_pandas()
simbad_df["IDS_ls"] = simbad_df["IDS"].apply(lambda x: x.split("|"))
# get the WDS ID for those stars
simbad_df["WDS"] = simbad_df["IDS_ls"].apply(lambda x: [i for i in x if "WDS " in i])
simbad_df["WDS"] = simbad_df["WDS"].apply(lambda x: x[0] if len(x) > 0 else None)
# get the HD ID for those stars
simbad_df["HD"] = simbad_df["IDS_ls"].apply(lambda x: [i for i in x if "HD " in i])
simbad_df["HD"] = simbad_df["HD"].apply(lambda x: x[0] if len(x) > 0 else None)
# get the BD ID for those stars
simbad_df["BD"] = simbad_df["IDS_ls"].apply(
    lambda x: [i for i in x if ("BD+" in i or "BD-" in i)]
)
simbad_df["BD"] = simbad_df["BD"].apply(lambda x: x[0] if len(x) > 0 else None)
# get the CD ID for those stars
simbad_df["CD"] = simbad_df["IDS_ls"].apply(
    lambda x: [i for i in x if ("CD+" in i or "CD-" in i)]
)
simbad_df["CD"] = simbad_df["CD"].apply(lambda x: x[0] if len(x) > 0 else None)
# get the G ID for those , (Giclas 1971)
simbad_df["G"] = simbad_df["IDS_ls"].apply(lambda x: [i for i in x if ("G " in i)])
simbad_df["G"] = simbad_df["G"].apply(lambda x: x[0] if len(x) > 0 else None)

# find the row is binary
binary_idx = simbad_df["OTYPE_N"] == "**"
print("Number of stars is indicated as binary on SIMBAD: ", np.sum(binary_idx))

# add Sirius and a bunch or other HIP to the list to resolve binary and pop those rows out
hip_id = [
    32349,  # Sirius
    54577,
    48665,
    96073,
    60444,
    54953,
    49861,
    67951,
    40167,
    41554,
    41554,
    77269,
    30925,
    33792,
    15883,
    118310,
    20157,
    103542,
    106208,
    55987,  # children only have Gaia DR3 ID
    104589,  # parallax on SIMBAD is wrong, but children have correct parallax
    32400,
]

# known case of binary star (not even sure if they are binary) that I cannot resolve manually
hip_id_to_exclude = [
    38870,  # nothing good nearby in Gaia DR3, maybe the star is too bright
    88198,  # in dense field, no way one can match the correct one
    94367,  # nothing good nearby in Gaia DR3, I dont think it is a binary star
    94926,  # nothing good nearby in Gaia DR3
    96691,  # nothing good nearby in Gaia DR3
    97879,  # nothing good nearby in Gaia DR3
    71394,  # nothing good nearby in Gaia DR3
    24711,  # binary also resolved by Hipparcos, so no need to look for binary companion
    24712,  # binary of the one above
    71394,  # binary also resolved by Hipparcos, so no need to look for binary companion
    71377,  # binary of the one above???
    31547,  # nothing good nearby in Gaia DR3
    31509,  # nothing good nearby in Gaia DR3
    116104,  # nothing good nearby in Gaia DR3
    115780,  # binary also resolved by Hipparcos, so no need to look for binary companion
    115781,  # binary of the one above
    19075,  # binary also resolved by Hipparcos, so no need to look for binary companion
    19076,  # binary of the one above
    20531,  # binary also resolved by Hipparcos, so no need to look for binary companion
    20533,  # binary of the one above
    12635,  # binary also resolved by Hipparcos, so no need to look for binary companion
    12638,  # binary of the one above
    10653,  # binary also resolved by Hipparcos, so no need to look for binary companion
    10660,  # binary of the one above
    39226,  # binary also resolved by Hipparcos, so no need to look for binary companion
    39228,  # binary of the one above
    75415,  # binary also resolved by Hipparcos, so no need to look for binary companion
    75411,  # binary of the one above
    110974,  # binary also resolved by Hipparcos, so no need to look for binary companion
    110976,  # binary of the one above
    31158,  # binary also resolved by Hipparcos, so no need to look for binary companion
    31156,  # binary of the one above
    26692,  # binary also resolved by Hipparcos, so no need to look for binary companion
    26690,  # binary of the one above
    20109,  # binary also resolved by Hipparcos, so no need to look for binary companion
    20074,  # binary of the one above
    8039,  # binary also resolved by Hipparcos, so no need to look for binary companion
    8038,  # binary of the one above
    46535,  # binary also resolved by Hipparcos, so no need to look for binary companion
    46523,  # binary of the one above
    42544,  # binary also resolved by Hipparcos, so no need to look for binary companion
    42543,  # binary of the one above
    39852,  # binary also resolved by Hipparcos, so no need to look for binary companion
    39867,  # binary of the one above
    71500,  # binary also resolved by Hipparcos, so no need to look for binary companion
    71502,  # binary of the one above
    110419,  # binary also resolved by Hipparcos, so no need to look for binary companion
    110433,  # binary of the one above
    110419,  # binary also resolved by Hipparcos, so no need to look for binary companion
    109368,  # binary of the one above
    17579,
    17588,
    107268,
    107253,
    110712,
    110719,
    95947,
    80170,
    13209,  # probably not a binary star
]

hip_lookup_idx = simbad_df["hip"].isin(hip_id)
hip_dont_lookup_idx = ~simbad_df["hip"].isin(hip_id_to_exclude)
final_hip_lookip_idx = (binary_idx | hip_lookup_idx) & hip_dont_lookup_idx
missing_df = simbad_df.loc[final_hip_lookip_idx].reset_index(drop=True)
print("Number of stars to look for binary companion: ", len(missing_df))


Number of stars is indicated as binary on SIMBAD:  8464
Number of stars to look for binary companion:  8428


In [2]:
custom_simbad = Simbad()
custom_simbad.TIMEOUT = 99999
custom_simbad.add_votable_fields(
    "ids",
    "ra(d2000)",
    "dec(d2000)",
    "pmra",
    "pmdec",
    "plx",
    "plx_error",
    "flux(U)",
    "flux(B)",
    "flux(V)",
    "flux(R)",
    "flux(I)",
    "flux(G)",
    "flux(J)",
    "flux(H)",
    "flux(K)",
    "otype(N)",
    "sptype",
    "rv_value",
    "rvz_error",
)


def extract_WDS_designations(designation):
    # Match letter groups at the end of the string, separated by commas
    matches = designation[15:]
    if "," in matches:
        matches = matches.split(",")
    else:
        # break up every letter
        matches = list(matches)
    # replace "Ba" with "A" and "Bb" with "B"
    if "Ba" in matches:
        matches[matches.index("Ba")] = "A"
    if "Bb" in matches:
        matches[matches.index("Bb")] = "B"
    if "BC" in matches:
        matches.remove("BC")
        matches.append("B")
        matches.append("C")
    return matches


# =============================================================================
# The problem is, it is difficult to say what is the designation of the star used by SIMBAD
# So use all the common ones used by them to check one by one
# =============================================================================
# loop their WDS id to query simbad
all_new_ids_to_query = []
for iidx, (hip, wds, hd, bd, cd, giclas) in enumerate(
    zip(
        missing_df["hip"],
        missing_df["WDS"],
        missing_df["HD"],
        missing_df["BD"],
        missing_df["CD"],
        missing_df["G"],
    )
):
    success_flag = False
    letters = None
    # place it here because it will success but without source ID
    if cd == "CD-40  6603":
        result_table = custom_simbad.query_object("CD-40 6603")
        result_table = vstack([result_table, result_table])
        result_table["IDS"] = [
            "HIP 54953|Gaia DR3 5395425092190193792",
            "HIP 54953|Gaia DR3 5395425092187016960",
        ]
        result_table["IDS"] = result_table["IDS"].astype(object)
        success_flag = True
    if wds == "WDS J14325+0547AB":
        result_table = custom_simbad.query_object("WDS J14325+0547AB")
        result_table = vstack([result_table, result_table])
        result_table["IDS"] = [
            "HIP 71108|Gaia DR3 3669289933632027904",
            "HIP 71108|Gaia DR3 3669289933632036736",
        ]
        result_table["IDS"] = result_table["IDS"].astype(object)
        success_flag = True
    if wds == "WDS J17372+2754AB":
        result_table = custom_simbad.query_object("WDS J17372+2754AB")
        result_table = vstack([result_table, result_table])
        result_table["IDS"] = [
            "HIP 86221|Gaia DR3 4595441941016988672",
            "HIP 86221|Gaia DR3 4595441868000475520",
        ]
        result_table["IDS"] = result_table["IDS"].astype(object)
        success_flag = True
    # WDS, there is an exception
    # HD 223248 bettwe resolve with HD id, something is wrong with its WDS
    if wds is not None and hd != "HD 223248" and not success_flag:
        letters = extract_WDS_designations(wds)
        if letters == ["A"]:  # so only A
            letters.append("B")
            letters.append("C")
            letters.append("BC")
        wds = wds[:15]
        if wds == "WDS J18563+5432":  # this one also cause trouble, do it manually
            wds = "MCC 806"
            letters = ["A", "B"]
        # combine wds to letters
        wds = [f"{wds}{j}" for j in letters]
        result_table = custom_simbad.query_objects(wds)
        if result_table is not None:
            success_flag = True
        else:
            # only WDS use Aa, Ab, Bb, Ba.... Prep for others below
            if "Aa" in letters:
                letters[letters.index("Aa")] = "A"
            if "Ab" in letters:
                letters[letters.index("Ab")] = "B"
    for sub_id in [hd, bd, cd, giclas]:
        if not success_flag and sub_id is not None:
            # if not then try up to D
            if letters is None or letters == ["A"]:
                letters = ["A", "B", "C", "D"]
            sub_id = [f"{sub_id}{j}" for j in letters]
            result_table = custom_simbad.query_objects(sub_id)
            if result_table is not None:
                success_flag = True
    if not success_flag:  # manually deal with known cases
        # only need to replace IDS because other data is going to be replaced anyway as long as having Gaia ID
        if cd == "CD-46  8293":
            result_table = custom_simbad.query_object("CD-46  8293")
            result_table = vstack([result_table, result_table])
            result_table["IDS"] = [
                "HIP 63301|Gaia DR3 6086002094754592256",
                "HIP 63301|Gaia DR3 6086002094759753856",
            ]
            result_table["IDS"] = result_table["IDS"].astype(object)
            success_flag = True
        if cd == "CD-49 11266":
            result_table = custom_simbad.query_object("CD-49 11266")
            result_table = vstack([result_table, result_table])
            result_table["IDS"] = [
                "HIP 84348|Gaia DR3 5926440108384499712",
                "HIP 84348|Gaia DR3 5926440108384499584",
            ]
            result_table["IDS"] = result_table["IDS"].astype(object)
            success_flag = True
    if not success_flag:  # only raise even after dealing with known cases
        print(f"Cannot find the star {wds}, {hd}, {bd}, {cd}, {giclas} at index {iidx}")
        result_table = None
    else:
        # change RAJ2000_d data type to float
        result_table["RA_d2000"] = result_table["RA_d2000"].filled(np.nan).astype(float)
        # change DECJ2000_d data type to float
        result_table["DEC_d2000"] = (
            result_table["DEC_d2000"].filled(np.nan).astype(float)
        )
        result_table["hip"] = hip
        # drop column RA and DEC
        result_table.remove_columns(["RA", "DEC"])
        if result_table["hip"][0] == 53765 or result_table["hip"][0] == 16324:  # I dunno why these star are problematic
            try:
                ra = np.array([result_table["RA_d2000"][0][0], result_table["RA_d2000"][0][1]])
                result_table["RA_d2000"] = ra
            except:
                pass
        if result_table["hip"][0] == 28832 or result_table["hip"][0] == 21230:  # I dunno why these star are problematic
            try:
                ra = np.array([result_table["DEC_d2000"][0][0], result_table["DEC_d2000"][0][1]])
                result_table["DEC_d2000"] = ra
            except:
                pass

        # check if the star is 'G 229-19' because SIMBAD has mis-matched the star
        # MCC 806A -> Gaia DR3 2146777521017790464 -> HIP 92952 A
        # MCC 806B -> Gaia DR3 2146777525315026688 -> HIP 92952 B
        # if "G 229-19" in result_table["MAIN_ID"]:
        #     # insert a row at the end
        #     result_table.add_row()
        #     result_table["MAIN_ID"] = ["MCC 806A", "MCC 806B"]
        #     result_table["IDS"] = [
        #         "HIP 92952|Gaia DR3 2146777521017790464",
        #         "HIP 92952|Gaia DR3 2146777525315026688",
        #     ]
        #     # turn it to object type
        #     result_table["MAIN_ID"] = result_table["MAIN_ID"].astype(object)
        #     result_table["IDS"] = result_table["IDS"].astype(object)
        all_new_ids_to_query.append(result_table)

# save the result table as a backup
result_table = vstack(all_new_ids_to_query, metadata_conflicts="silent")
# these columns are not needed and cause saving error
# result_table.remove_columns(["DESIGNATION", "phot_variable_flag", "libname_gspphot", "random_index"])
result_table.write(
    base_path / "hip_binary_crossmatch.dat", format="ascii", overwrite=True
)

Cannot find the star ['WDS J23304+3050A', 'WDS J23304+3050B'], HD 221264, BD+30  4963, None, None at index 0


Cannot find the star ['WDS J23313-5400A', 'WDS J23313-5400B'], HD 221285, None, CD-54  9556, None at index 7
Cannot find the star ['WDS J23319+7317A', 'WDS J23319+7317B'], None, BD+72  1107, None, None at index 10
Cannot find the star ['WDS J23319+4733A', 'WDS J23319+4733B'], None, BD+46  4070, None, None at index 11
Cannot find the star ['WDS J23322+0705A', 'WDS J23322+0705B'], HD 221445, BD+06  5168, None, None at index 12
Cannot find the star ['WDS J23326-4520A', 'WDS J23326-4520B'], HD 221465, None, CD-46 14689, None at index 14


Cannot find the star ['WDS J23338-0508A', 'WDS J23338-0508B'], HD 221630, BD-05  6012, None, None at index 16


Cannot find the star ['WDS J23350+0136A', 'WDS J23350+0136B', 'WDS J23350+0136C'], None, BD+00  5017, None, G  29-47 at index 19
Cannot find the star ['WDS J23363-0707A', 'WDS J23363-0707B'], HD 221925, BD-07  6055, None, None at index 26
Cannot find the star ['WDS J23364-4342A', 'WDS J23364-4342B'], HD 221918, None, CD-44 15292, None at index 28


Cannot find the star ['WDS J23365-6245A', 'WDS J23365-6245B'], HD 221910, None, CD-63  1614, None at index 29
Cannot find the star ['WDS J23374+0737A', 'WDS J23374+0737B'], HD 222068, BD+06  5178, None, None at index 33
Cannot find the star ['WDS J23378+6601A', 'WDS J23378+6601B'], None, BD+65  1920, None, None at index 36


...

Cannot find the star ['WDS J21477-3054A', 'WDS J21477-3054B'], HD 207155, None, CD-31 18466, None at index 8391
Cannot find the star ['WDS J21477-1813A', 'WDS J21477-1813B'], HD 207208, BD-18  6013, None, None at index 8392
Cannot find the star ['WDS J21478+6203A', 'WDS J21478+6203B'], HD 207577, BD+61  2199, None, None at index 8393
Cannot find the star ['WDS J21478-7241A', 'WDS J21478-7241B'], HD 206692, None, CD-73  1582, None at index 8394


Cannot find the star ['WDS J21481-3839A', 'WDS J21481-3839B'], HD 207156, None, CD-39 14434, None at index 8396
Cannot find the star ['WDS J21492-3538A', 'WDS J21492-3538B'], HD 207363, None, CD-36 15029, None at index 8402
Cannot find the star ['WDS J21496-0819A', 'WDS J21496-0819B'], HD 207502, BD-08  5746, None, None at index 8403
Cannot find the star ['WDS J21496+3319A', 'WDS J21496+3319B'], HD 207623, BD+32  4269, None, None at index 8404


Cannot find the star ['WDS J21498+3455A', 'WDS J21498+3455B'], HD 207663, BD+34  4540, None, None at index 8405
Cannot find the star ['WDS J21499-2509A', 'WDS J21499-2509B'], HD 207477, None, CD-25 15601, None at index 8406
Cannot find the star ['WDS J21504-5818A', 'WDS J21504-5818B'], HD 207377, None, None, None at index 8407
Cannot find the star ['WDS J21507+7746A', 'WDS J21507+7746B'], HD 208391, BD+77   836, None, None at index 8409
Cannot find the star ['WDS J21509-5939A', 'WDS J21509-5939B'], HD 207415, None, CD-60  7794, None at index 8411


Cannot find the star ['WDS J21510+2911A', 'WDS J21510+2911B'], HD 207794, BD+28  4210, None, None at index 8414
Cannot find the star ['WDS J21511+4711A', 'WDS J21511+4711B'], HD 207886, BD+46  3463, None, None at index 8416
Cannot find the star ['WDS J21511+6650A', 'WDS J21511+6650B'], HD 208074, BD+66  1446, None, None at index 8418
Cannot find the star ['WDS J21511+1022A', 'WDS J21511+1022B'], HD 207758, BD+09  4913, None, None at index 8419


Cannot find the star ['WDS J21518+5534A', 'WDS J21518+5534B'], HD 235643, BD+54  2635, None, None at index 8423
Cannot find the star ['WDS J21520+3826A', 'WDS J21520+3826B'], HD 207955, BD+37  4441, None, None at index 8424
Cannot find the star ['WDS J21523-7033A', 'WDS J21523-7033B'], HD 207430, None, CD-71  1708, None at index 8426


In [3]:
# parse the result
hip_to_delete = []  # because they are resolved as binary

for iidx, row_table in enumerate(all_new_ids_to_query):
    if row_table is not None and len(row_table) > 0:
        row_table["IDS_ls"] = [i.split("|") for i in row_table["IDS"]]
        row_table["source_id"] = 0
        fail_flag = 0
        for jidex, x in enumerate(
            row_table["IDS_ls"]
        ):  # try to resolve Gaia DR3 ID from SIMBAD
            if y := [y for y in x if "Gaia DR3" in y]:
                if len(y) > 0:
                    row_table["source_id"][jidex] = y[0][9:]
        if np.any(
            row_table["source_id"][1:] == 0
        ):  # if any sub-component still not having source_id, try to resolve it
            for j in range(1, len(row_table)):
                result = Gaia.cone_search(
                    row_table["MAIN_ID"][j], radius=2 * u.arcsec
                ).get_results()
                # exclude the star source_id already in the row_table
                result = result[~np.isin(result["SOURCE_ID"], row_table["source_id"])]
                if len(result) == 0:
                    break
                # select the brightest one, because either those stars are too dim to be observed by Hipparcos
                # or they are in binary and HIP refers to the main one anyway
                result = result[
                    result["phot_g_mean_mag"] == np.min(result["phot_g_mean_mag"])
                ]
                assert len(result) == 1
                # check if the campanion is too dim to be deal with
                if result["phot_g_mean_mag"] > 15 or np.isnan(
                    result["phot_g_mean_mag"]
                ):
                    print(f"{iidx} has a too dim companion or missing magnitude")
                    fail_flag = 1
                    break
                else:
                    row_table["source_id"][j] = result[
                        "SOURCE_ID"
                    ]  # otherwise add the source_id
                    row_table["FLUX_G"][j] = result[
                        "phot_g_mean_mag"
                    ]  # add the G magnitude
        # check if all sub-components are resolved and have Gaia DR3 ID, if not, pretend they are not binary because a lot of manual work is needed
        # It is okay if the main star has no source_id if too bright (e.g., Sirius and Sirius B)
        # check all source_id is unique, if not don't bother
        if row_table["FLUX_V"][0] < 4.0:
            _row_table_not_main = row_table[1:]
            source_ids_okay = np.all(_row_table_not_main["source_id"] != 0) & (
                np.all(_row_table_not_main["FLUX_G"].filled(np.nan) < 15)
            )
        else:
            source_ids_okay = np.all(row_table["source_id"] != 0) & (
                np.all(
                    row_table["FLUX_G"][row_table["source_id"] != 0].filled(np.nan) < 15
                )
            )
        if source_ids_okay and len(np.unique(row_table["source_id"])) == len(row_table):
            # if all resolved, then delete the HIP ID
            hip_to_delete.append(row_table["hip"][0])
            row_table["componentid"] = np.arange(1, len(row_table) + 1)
            row_table.remove_column("IDS_ls")
        else:
            print(f"{iidx} has no good Gaia source ID")
            fail_flag += 1
    else:  # because row_table only has 1 row, so it is not resolved as binary and better to use original entry
        fail_flag = 1
    if fail_flag > 0:
        all_new_ids_to_query[iidx] = None

0 has no good Gaia source ID
1 has no good Gaia source ID
5 has no good Gaia source ID
6 has no good Gaia source ID
10 has no good Gaia source ID
15 has no good Gaia source ID
16 has no good Gaia source ID
22 has no good Gaia source ID
23 has no good Gaia source ID
26 has no good Gaia source ID
29 has no good Gaia source ID
35 has no good Gaia source ID
43 has no good Gaia source ID
62 has no good Gaia source ID
65 has no good Gaia source ID
70 has no good Gaia source ID
80 has no good Gaia source ID
82 has no good Gaia source ID
83 has no good Gaia source ID
87 has no good Gaia source ID
88 has no good Gaia source ID
89 has no good Gaia source ID
90 has no good Gaia source ID
91 has no good Gaia source ID
93 has no good Gaia source ID
94 has no good Gaia source ID
101 has no good Gaia source ID
106 has no good Gaia source ID
110 has no good Gaia source ID
111 has no good Gaia source ID
114 has no good Gaia source ID
115 has no good Gaia source ID
116 has no good Gaia source ID
117 has

In [4]:
# delete those hip from simbad_df
simbad_df = simbad_df[~simbad_df["hip"].isin(hip_to_delete)].reset_index(drop=True)
binary_t = vstack(
    [i for i in all_new_ids_to_query if i is not None], metadata_conflicts="silent"
)
# check all source_id is unique, if not something is wrong
assert len(np.unique(binary_t["source_id"][binary_t["source_id"] != 0])) == len(
    binary_t[binary_t["source_id"] != 0]
)
# dtop the columns that are not needed
simbad_df.drop(
    columns=[
        "WDS",
        "HD",
        "CD",
        "BD",
        "G",
        "IDS_ls",
    ],
    inplace=True,
)

# add the new data to the main dataframe
simbad_df = pd.concat([simbad_df, binary_t.to_pandas()], ignore_index=True)

# check all source_id is unique, if not something is wrong
assert len(np.unique(simbad_df["source_id"][simbad_df["source_id"] != 0])) == len(
    simbad_df[simbad_df["source_id"] != 0]
)

# save the table using astropy Table
simbad_df = Table.from_pandas(simbad_df)
# make sure overwrite is False to not overwrite the original data file
simbad_df.write(
    base_path / "hip_processed_with_binary.dat", format="ascii", overwrite=True
)